In [ ]:
import os
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from umap import UMAP
import torch
from torch import nn
import optuna

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.manual_seed(0)

In [ ]:
# Import custom modules
from utils.data_utils import construct_loader
from utils.model_utils import GNN
from utils.train_test_utils import (
    Standardizer,
    train_epoch,
    train_epoch_without_SSL,
    pred,
    pred_with_TTA,
    embeddings_with_TTA,
)

# Training

In [ ]:
batch_size = 512
epochs = 20

train_hyperparam_opt = False # Optional optimization of training and testing hyperparameters
test_hyperparam_opt = False # Optional optimization of testing hyperparameters

# If wanted, we can load an already trained model
save_model = False  # Set to False if you don't want to save
load_trained_model = True  # Set to True to skip training and load existing model
save_plots = False

### Loading and preprocessing the Data

In [ ]:
data_df = pd.read_csv("data/AqSolDBc.csv")
# Drop single atoms
idx_single = [
    i
    for i, s in enumerate(data_df["SmilesCurated"])
    if Chem.MolFromSmiles(s).GetNumAtoms() == 1 or "." in s
]

data_df = data_df.drop(idx_single)
if len(idx_single) > 0:
    print(f"Removing {idx_single} due to single atoms")

test_df = pd.read_csv("data/OChemUnseen.csv")
# Drop some Nonetypes
# Got an error for a SMILES which was None
idx_nonetype = [
    i for i, s in enumerate(test_df["SMILES"]) if Chem.MolFromSmiles(s) is None
]

test_df = test_df.drop(idx_nonetype)
if len(idx_nonetype) > 0:
    print(f"Removing {idx_nonetype} due to Nonetypes")

# Drop single atoms
idx_single = [
    i
    for i, s in enumerate(test_df["SMILES"])
    if Chem.MolFromSmiles(s).GetNumAtoms() == 1 or "." in s
]

test_df = test_df.drop(idx_single)
if len(idx_single) > 0:
    print(f"Removing {idx_single} due to single atoms")

In [ ]:
train_df, val_df = train_test_split(data_df, test_size=0.1, random_state=0)
train_loader = construct_loader(
    train_df, "SmilesCurated", "ExperimentalLogS", shuffle=True, batch_size=batch_size
)

val_loader = construct_loader(
    val_df, "SmilesCurated", "ExperimentalLogS", shuffle=False, batch_size=batch_size
)

test_loader = construct_loader(test_df, "SMILES", "LogS", shuffle=False, batch_size=1)
print(
    f"Train size: {len(train_loader.dataset)}, Val size: {len(val_loader.dataset)}, Test size: {len(test_loader.dataset)}"
)

# Standardizer for the solubility labels
mean = np.mean(train_loader.dataset.labels)
std = np.std(train_loader.dataset.labels)
stdzer = Standardizer(mean, std)

### Hyperparameter optimization

In [ ]:
# Optional hyperparameter optimization on the validation set


def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 5e-3, log=True)
    # Weighting the losses of the two tasks
    alpha = trial.suggest_float("alpha", 0.3, 0.6, step=0.1)  
    # Size of the encoder-decoder bottleneck
    hidden_size = trial.suggest_categorical("hidden_size", [16, 32])  
    depth = trial.suggest_categorical("depth", [3, 4, 5])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.2, 0.3])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 5e-5, log=True)

    model = GNN(
        train_loader.dataset.num_node_features,
        train_loader.dataset.num_edge_features,
        hidden_size=hidden_size,
        depth=depth,
        dropout=dropout,
    ).to(device)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=weight_decay
    )

    loss = nn.MSELoss(reduction="mean")

    # By default, we train for a fourth of the epochs used in the main training to get a rough estimate
    for epoch in range(epochs // 2):
        train_epoch(model, train_loader, optimizer, loss, alpha, stdzer)

        preds = pred(model, val_loader, mode="predict", stdzer=stdzer)
        rmse_pred = root_mean_squared_error(preds, val_loader.dataset.labels)

        trial.report(rmse_pred, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return rmse_pred

In [ ]:
# Takes about half an hour on my machine for a setting of 20 epochs
if train_hyperparam_opt:
    study = optuna.create_study(
        direction="minimize",
        pruner=optuna.pruners.HyperbandPruner(),
        study_name="hyperparam_opt",
    )
    
    study.optimize(objective, n_trials=40, n_jobs=4)

    print(f"Best learning rate: {study.best_params['learning_rate']:.4f}")
    print(f"Best alpha: {study.best_params['alpha']:.4f}")
    print(f"Best hidden size: {study.best_params['hidden_size']:.4f}")
    print(f"Best depth: {study.best_params['depth']:.4f}")
    print(f"Best dropout: {study.best_params['dropout']:.4f}")
    print(f"Best weight decay: {study.best_params['weight_decay']:.4f}")

    study_results = study.trials_dataframe()
    study_results.to_csv("data/hyperparam_opt.csv", index=False)
    print("Study results saved to hyperparam_opt.csv")

    learning_rate = study.best_params["learning_rate"]
    alpha = study.best_params["alpha"]
    hidden_size = study.best_params["hidden_size"]
    depth = study.best_params["depth"]
    dropout = study.best_params["dropout"]
    weight_decay = study.best_params["weight_decay"]

    torch.cuda.empty_cache()

# Standard hyperparameters are the result of a previous hyperparameter optimization
else:
    learning_rate = 0.0026
    alpha = 0.3  # Weighting the losses of the two tasks, smaller alpha means less weight on the denoising task
    hidden_size = 32
    depth = 3
    dropout = 0.3
    weight_decay = 5e-6

### Training on both Tasks

In [ ]:
model = GNN(
    train_loader.dataset.num_node_features,
    train_loader.dataset.num_edge_features,
    hidden_size=hidden_size,
    depth=depth,
    dropout=dropout,
).to(device)

optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay
)

loss = nn.MSELoss(reduction="mean")
print("\n", model, "\n")

In [ ]:
# If wanted, we can load an already trained model

# Path for the best model
model_path = os.path.join("trained_models", "model.pt")
if load_trained_model and os.path.exists(model_path):
    # Load pretrained model
    print(f"Loading model from {model_path}")
    model.load_state_dict(torch.load(model_path))
    best_model = deepcopy(model).to(device)
    # Skip training
    epochs = 0

In [ ]:
# Training the model on both tasks simultaneously
if not load_trained_model:
    best_model = deepcopy(model).to(device)
    best_combined_val_loss = 1e5

    combined_train_loss_list = []
    denoise_train_loss_list = []
    pred_train_loss_list = []

    combined_val_loss_list = []
    denoise_val_loss_list = []
    pred_val_loss_list = []

    if epochs == 0:
        print("Skipping training, previously trained model was loaded.")

    for epoch in range(0, epochs):
        combined_train_loss, denoise_train_loss, pred_train_loss = train_epoch(
            model, train_loader, optimizer, loss, alpha=alpha, stdzer=stdzer
        )

        denoised = pred(model, val_loader, mode="denoise", stdzer=stdzer)
        node_feature_targets = [
            feature
            for batch in val_loader
            for feature in batch.x.cpu().flatten().tolist()
        ]
        
        edge_feature_targets = [
            feature
            for batch in val_loader
            for feature in batch.edge_attr.cpu().flatten().tolist()
        ]

        denoise_val_loss = root_mean_squared_error(
            denoised, node_feature_targets + edge_feature_targets
        )

        preds = pred(model, val_loader, mode="predict", stdzer=stdzer)
        pred_val_loss = root_mean_squared_error(preds, val_loader.dataset.labels)
        combined_val_loss = alpha * denoise_val_loss + (1 - alpha) * pred_val_loss

        print(
            f"Epoch {epoch}  Train Combined Loss: {combined_train_loss:.3f}  Train Denoise Loss: {denoise_train_loss:.3f}  Train Pred Loss: {pred_train_loss:.3f}  Val Combined Loss: {combined_val_loss:.3f}  Val Denoise Loss: {denoise_val_loss:.3f}  Val Pred Loss: {pred_val_loss:.3f}"
        )

        combined_train_loss_list.append(combined_train_loss)
        denoise_train_loss_list.append(denoise_train_loss)
        pred_train_loss_list.append(pred_train_loss)
        denoise_val_loss_list.append(denoise_val_loss)
        pred_val_loss_list.append(pred_val_loss)
        combined_val_loss_list.append(combined_val_loss)


        if combined_val_loss < best_combined_val_loss:
            best_model = deepcopy(model).to(device)
            best_combined_val_loss = combined_val_loss

            # Save the best model
            if save_model:
                print(f"Saving best model based on combined Val Loss...")
                torch.save(
                    best_model.state_dict(), os.path.join("trained_models", f"model.pt")
                )

In [ ]:
# Only plot if actual training was done
if epochs != 0:

    def normalize_losses(loss_list):
        # Normalize losses to [0, 1]
        min_val, max_val = min(loss_list), max(loss_list)
        return [(loss - min_val) / (max_val - min_val) for loss in loss_list]

    combined_train_loss_list_norm = normalize_losses(combined_train_loss_list)
    denoise_train_loss_list_norm = normalize_losses(denoise_train_loss_list)
    pred_train_loss_list_norm = normalize_losses(pred_train_loss_list)
    combined_val_loss_list_norm = normalize_losses(combined_val_loss_list)
    denoise_val_loss_list_norm = normalize_losses(denoise_val_loss_list)
    pred_val_loss_list_norm = normalize_losses(pred_val_loss_list)

    fig, ax = plt.subplots(1, 1, figsize=(8, 4))

    # Plot normalized losses
    ax.plot(
        list(range(epochs)),
        combined_train_loss_list_norm,
        label="Combined Train Loss",
        color="blue",
    )
    ax.plot(
        list(range(epochs)),
        denoise_train_loss_list_norm,
        label="Denoise Train Loss",
        color="orange",
    )
    ax.plot(
        list(range(epochs)),
        pred_train_loss_list_norm,
        label="Pred Train Loss",
        color="green",
    )
    ax.plot(
        list(range(epochs)),
        combined_val_loss_list_norm,
        label="Combined Val Loss",
        color="blue",
        linestyle="dashed",
    )
    ax.plot(
        list(range(epochs)),
        denoise_val_loss_list_norm,
        label="Denoise Val Loss",
        color="orange",
        linestyle="dashed",
    )
    ax.plot(
        list(range(epochs)),
        pred_val_loss_list_norm,
        label="Pred Val Loss",
        color="green",
        linestyle="dashed",
    )
    
    ax.legend(ncol=2)
    ax.set_xlabel("Epochs")
    ax.set_xticks(list(range(0, epochs, 5)))
    ax.set_ylabel("Normalized Loss")

    plt.tight_layout()
    if save_plots:
        plt.savefig("figures/loss_plot.jpg", dpi=300, bbox_inches="tight")
        plt.savefig("figures/loss_plot.pdf", bbox_inches="tight")
    plt.show()

### Training without SSL for reference

In [ ]:
model_non_SSL = GNN(
    train_loader.dataset.num_node_features,
    train_loader.dataset.num_edge_features,
    hidden_size=hidden_size,
    depth=depth,
    dropout=dropout,
).to(device)

optimizer_non_SSL = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay
)

loss_non_SSL = nn.MSELoss(reduction="mean")
print("\n", model, "\n")

In [ ]:
# If wanted, we can load an already trained model

# Path for the best model
model_path = os.path.join("trained_models", "model_non_SSL.pt")
if load_trained_model and os.path.exists(model_path):
    # Load pretrained model
    print(f"Loading model from {model_path}")
    model_non_SSL.load_state_dict(torch.load(model_path))
    best_model_non_SSL = deepcopy(model_non_SSL).to(device)
    # Skip training
    epochs = 0

In [ ]:
best_model_non_SSL = deepcopy(model_non_SSL).to(device)
best_val_loss = 1e5

train_loss_list = []
val_loss_list = []

if epochs == 0:
    print("Skipping training, previously trained model was loaded.")

for epoch in range(0, epochs):
    train_loss = train_epoch_without_SSL(
        model_non_SSL,
        train_loader,
        optimizer_non_SSL,
        loss_non_SSL,
        alpha=alpha,
        stdzer=stdzer,
    )

    preds = pred(model_non_SSL, val_loader, mode="predict", stdzer=stdzer)
    val_loss = root_mean_squared_error(preds, val_loader.dataset.labels)

    print(f"Epoch {epoch}  Train Loss: {train_loss:.2f}  Val Loss: {val_loss:.2f}")

    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

    if val_loss < best_val_loss:
        best_model_non_SSL = deepcopy(model_non_SSL).to(device)
        best_val_loss = val_loss

        # Save the best model
        if save_model:
            print(f"Saving best model based on Val Loss...")
            torch.save(
                best_model_non_SSL.state_dict(),
                os.path.join("trained_models", f"model_non_SSL.pt"),
            )

# Prediction

In [ ]:
# Optional hyperparameter optimization on the validation set for the TTA step

def objective(trial):
    learning_rate_TTA = trial.suggest_float("learning_rate_TTA", 1e-5, 1.0, log=True)

    preds_TTA = pred_with_TTA(
        best_model, val_loader, lr=learning_rate_TTA, stdzer=stdzer
    )

    return root_mean_squared_error(preds_TTA, val_loader.dataset.labels)

In [ ]:
if test_hyperparam_opt:
    study = optuna.create_study(
        direction="minimize",
        pruner=optuna.pruners.HyperbandPruner(),
        study_name="TTA_hyperparam_opt",
    )
    
    study.optimize(objective, n_trials=30, n_jobs=6)

    print(f"Best TTA learning rate: {study.best_params['learning_rate_TTA']:.5f}")

    study_results = study.trials_dataframe()
    study_results.to_csv("data/TTA_hyperparam_opt.csv", index=False)
    print("Study results saved to TTA_hyperparam_opt.csv")

    learning_rate_TTA = study.best_params["learning_rate_TTA"]

# Standard hyperparameters are the result of a previous hyperparameter optimization
else:
    learning_rate_TTA = 0.00021  # It seems as the best TTA learning rate is about a tenth of the training learning rate

In [ ]:
# Prediction with non-SSL model
val_preds_non_SSL = pred(best_model_non_SSL, val_loader, mode="predict", stdzer=stdzer)
test_preds_non_SSL = pred(
    best_model_non_SSL, test_loader, mode="predict", stdzer=stdzer
)

print("Reference results for model trained without SSL:")
print(
    f"Val Set RMSE: {root_mean_squared_error(val_preds_non_SSL, val_loader.dataset.labels):.4f}"
)
print(
    f"Test Set RMSE: {root_mean_squared_error(test_preds_non_SSL, test_loader.dataset.labels):.4f}"
)

In [ ]:
# Prediction with SLL model without TTA
val_preds = pred(best_model, val_loader, mode="predict", stdzer=stdzer)
test_preds = pred(best_model, test_loader, mode="predict", stdzer=stdzer)
print("Results without TTA:")
print(
    f"Val Set RMSE: {root_mean_squared_error(val_preds, val_loader.dataset.labels):.4f}"
)
print(
    f"Test Set RMSE: {root_mean_squared_error(test_preds, test_loader.dataset.labels):.4f}"
)

In [ ]:
# Prediction with SLL model with TTA
# Takes about three times as long as prediction without TTA
val_preds_TTA = pred_with_TTA(
    best_model, val_loader, lr=learning_rate_TTA, stdzer=stdzer
)
test_preds_TTA = pred_with_TTA(
    best_model, test_loader, lr=learning_rate_TTA, stdzer=stdzer
)

print("Results on with TTA:")
print(
    f"Val Set RMSE: {root_mean_squared_error(val_preds_TTA, val_loader.dataset.labels):.4f}"
)
print(
    f"Test Set RMSE: {root_mean_squared_error(test_preds_TTA, test_loader.dataset.labels):.4f}"
)

# Analysis

### Getting the Embeddings

In [ ]:
best_model = best_model.to(device)

# The embeddings are lists of length "samples" of vectors with length "hidden_size"
train_embeddings = []
for batch in train_loader:
    batch = batch.to(device)
    embedding = best_model.get_embedding(batch)
    train_embeddings.extend(embedding.cpu().detach().numpy())

val_embeddings = []
for batch in val_loader:
    batch = batch.to(device)
    embedding = best_model.get_embedding(batch)
    val_embeddings.extend(embedding.cpu().detach().numpy())

test_embeddings = []
for batch in test_loader:
    batch = batch.to(device)
    embedding = best_model.get_embedding(batch)
    test_embeddings.extend(embedding.cpu().detach().numpy())

# Combine all embeddings for t-SNE fitting
all_embeddings = np.vstack([train_embeddings, val_embeddings, test_embeddings])

# Create and fit t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
all_embeddings_tsne = tsne.fit_transform(all_embeddings)

# Split back into individual datasets
train_size = len(train_embeddings)
val_size = len(val_embeddings)
test_size = len(test_embeddings)

train_embeddings_tsne = all_embeddings_tsne[:train_size]
val_embeddings_tsne = all_embeddings_tsne[train_size : train_size + val_size]
test_embeddings_tsne = all_embeddings_tsne[train_size + val_size :]

# Create and fit UMAP for 2D projections
umap_reducer = UMAP(
    n_components=2,
)
all_embeddings_umap = umap_reducer.fit_transform(all_embeddings, n_jobs=-1)

# Split UMAP results back into individual datasets
train_embeddings_umap = all_embeddings_umap[:train_size]
val_embeddings_umap = all_embeddings_umap[train_size : train_size + val_size]
test_embeddings_umap = all_embeddings_umap[train_size + val_size :]

# Also getting the test set embeddings after TTA
test_embeddings_with_TTA = embeddings_with_TTA(
    best_model, test_loader, lr=learning_rate_TTA
)

test_embeddings_with_TTA_tsne = tsne.fit_transform(np.array(test_embeddings_with_TTA))
test_embeddings_with_TTA_umap = umap_reducer.transform(
    np.array(test_embeddings_with_TTA)
)

### Analyzing the Solubility Distribution

In [ ]:
# Get the solubility values from each dataset
train_solubility = train_loader.dataset.labels
val_solubility = val_loader.dataset.labels
test_solubility = test_loader.dataset.labels

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Define a consistent colormap for solubility
cmap = plt.cm.viridis
norm = plt.Normalize(
    min(min(train_solubility), min(val_solubility), min(test_solubility)),
    max(max(train_solubility), max(val_solubility), max(test_solubility)),
)

# Plot t-SNE projection on the left
sc1 = ax1.scatter(
    train_embeddings_tsne[:, 0],
    train_embeddings_tsne[:, 1],
    c=train_solubility,
    cmap=cmap,
    norm=norm,
    s=1.5,
)
ax1.scatter(
    val_embeddings_tsne[:, 0],
    val_embeddings_tsne[:, 1],
    c=val_solubility,
    cmap=cmap,
    norm=norm,
    s=1.5,
)
ax1.scatter(
    test_embeddings_tsne[:, 0],
    test_embeddings_tsne[:, 1],
    c=test_solubility,
    cmap=cmap,
    norm=norm,
    s=1.5,
)

ax1.set_title("t-SNE Projection")
ax1.set_xlabel("t-SNE Component 1")
ax1.set_ylabel("t-SNE Component 2")

# Plot PCA projection on the right
sc2 = ax2.scatter(
    train_embeddings_umap[:, 0],
    train_embeddings_umap[:, 1],
    c=train_solubility,
    cmap=cmap,
    norm=norm,
    s=1.5,
)
ax2.scatter(
    val_embeddings_umap[:, 0],
    val_embeddings_umap[:, 1],
    c=val_solubility,
    cmap=cmap,
    norm=norm,
    s=1.5,
)
ax2.scatter(
    test_embeddings_umap[:, 0],
    test_embeddings_umap[:, 1],
    c=test_solubility,
    cmap=cmap,
    norm=norm,
    s=1.5,
)

ax2.set_title("UMAP Projection")
ax2.set_xlabel("UMAP Component 1")
ax2.set_ylabel("UMAP Component 2")

# Dynamically adjust the limits for both plots
x_min_train_tsne, x_max_train_tsne = np.percentile(train_embeddings_tsne[:, 0], [5, 95])
y_min_train_tsne, y_max_train_tsne = np.percentile(train_embeddings_tsne[:, 1], [5, 95])
x_min_test_tsne, x_max_test_tsne = np.percentile(test_embeddings_tsne[:, 0], [5, 95])
y_min_test_tsne, y_max_test_tsne = np.percentile(test_embeddings_tsne[:, 1], [5, 95])

x_max_tsne = max(x_max_train_tsne, x_max_test_tsne)
x_min_tsne = min(x_min_train_tsne, x_min_test_tsne)
y_max_tsne = max(y_max_train_tsne, y_max_test_tsne)
y_min_tsne = min(y_min_train_tsne, y_min_test_tsne)

ax1.set_xlim(x_min_tsne, x_max_tsne)
ax1.set_ylim(y_min_tsne, y_max_tsne)

x_min_train_umap, x_max_train_umap = np.percentile(train_embeddings_umap[:, 0], [5, 95])
y_min_train_umap, y_max_train_umap = np.percentile(train_embeddings_umap[:, 1], [5, 95])
x_min_test_umap, x_max_test_umap = np.percentile(test_embeddings_umap[:, 0], [5, 95])
y_min_test_umap, y_max_test_umap = np.percentile(test_embeddings_umap[:, 1], [5, 95])

x_max_umap = max(x_max_train_umap, x_max_test_umap)
x_min_umap = min(x_min_train_umap, x_min_test_umap)
y_max_umap = max(y_max_train_umap, y_max_test_umap)
y_min_umap = min(y_min_train_umap, y_min_test_umap)

ax2.set_xlim(x_min_umap, x_max_umap)
ax2.set_ylim(y_min_umap, y_max_umap)

fig.tight_layout()
cbar = fig.colorbar(sc2, ax=[ax2], aspect=50, pad=0.015)
cbar.set_label("Solubility")

if save_plots:
    plt.savefig("figures/solubility.jpg", dpi=300, bbox_inches="tight")
    plt.savefig("figures/solubility.pdf", bbox_inches="tight")
plt.show()

### Analyzing the Class Distribution and Effect of TTA

In [ ]:
# Create 2D embeddings using both t-SNE and PCA for the plot
train_embeddings_2d = train_embeddings_umap
val_embeddings_2d = val_embeddings_umap
test_embeddings_2d = test_embeddings_umap
test_embeddings_with_TTA_2d = test_embeddings_with_TTA_umap

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# t-SNE plot on the left
ax1.scatter(
    train_embeddings_tsne[:, 0],
    train_embeddings_tsne[:, 1],
    label="Train Set",
    s=1.5,
    color="blue",
)
ax1.scatter(
    val_embeddings_tsne[:, 0],
    val_embeddings_tsne[:, 1],
    label="Validation Set",
    s=1.5,
    color="orange",
)
ax1.scatter(
    test_embeddings_tsne[:, 0],
    test_embeddings_tsne[:, 1],
    label="Test Set",
    s=1.5,
    color="green",
)
ax1.scatter(
    test_embeddings_with_TTA_tsne[:, 0],
    test_embeddings_with_TTA_tsne[:, 1],
    label="Test Set with TTA",
    s=1.5,
    color="red",
)

# Calculate centroids
train_centroid_tsne = np.mean(train_embeddings_tsne, axis=0)
val_centroid_tsne = np.mean(val_embeddings_tsne, axis=0)
test_centroid_tsne = np.mean(test_embeddings_tsne, axis=0)
test_tta_centroid_tsne = np.mean(test_embeddings_with_TTA_tsne, axis=0)


ax1.set_title("t-SNE Projection")
ax1.set_xlabel("t-SNE Component 1")
ax1.set_ylabel("t-SNE Component 2")
ax1.set_xlim(x_min_tsne, x_max_tsne)
ax1.set_ylim(y_min_tsne, y_max_tsne)
fig.legend(bbox_to_anchor=(0.75, -0.05), loc="lower right", ncol=4)

# PCA plot on the right
ax2.scatter(
    train_embeddings_2d[:, 0],
    train_embeddings_2d[:, 1],
    label="Train Set",
    s=1.5,
    color="blue",
)
ax2.scatter(
    val_embeddings_2d[:, 0],
    val_embeddings_2d[:, 1],
    label="Validation Set",
    s=1.5,
    color="orange",
)
ax2.scatter(
    test_embeddings_2d[:, 0],
    test_embeddings_2d[:, 1],
    label="Test Set",
    s=1.5,
    color="green",
)
ax2.scatter(
    test_embeddings_with_TTA_2d[:, 0],
    test_embeddings_with_TTA_2d[:, 1],
    label="Test Set with TTA",
    s=1.5,
    color="red",
)

# Calculate centroids with median instead of mean since we have outliers
train_centroid = np.median(train_embeddings_2d, axis=0)
val_centroid = np.median(val_embeddings_2d, axis=0)
test_centroid = np.median(test_embeddings_2d, axis=0)
test_tta_centroid = np.median(test_embeddings_with_TTA_2d, axis=0)

ax2.set_title("UMAP Projection")
ax2.set_xlabel("UMAP Component 1")
ax2.set_ylabel("UMAP Component 2")
ax2.set_xlim(x_min_umap, x_max_umap)
ax2.set_ylim(y_min_umap, y_max_umap)

plt.tight_layout()
if save_plots:
    plt.savefig("figures/sets.jpg", dpi=300, bbox_inches="tight")
    plt.savefig("figures/sets.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# t-SNE plot on the left
ax1.scatter(
    test_embeddings_tsne[:, 0],
    test_embeddings_tsne[:, 1],
    alpha=0.5,
    label="Test Set without TTA",
    s=1.5,
    color="green",
)
ax1.scatter(
    test_embeddings_with_TTA_tsne[:, 0],
    test_embeddings_with_TTA_tsne[:, 1],
    alpha=0.5,
    label="Test Set with TTA",
    s=1.5,
    color="red",
)

# Plot centroids for t-SNE
ax1.scatter(
    train_centroid_tsne[0],
    train_centroid_tsne[1],
    s=250,
    c="blue",
    marker="P",
    edgecolors="black",
    linewidths=1,
    label="Train Centroid",
)
ax1.scatter(
    val_centroid_tsne[0],
    val_centroid_tsne[1],
    s=250,
    c="orange",
    marker="X",
    edgecolors="black",
    linewidths=1,
    label="Val Centroid",
)
ax1.scatter(
    test_centroid_tsne[0],
    test_centroid_tsne[1],
    s=250,
    c="green",
    marker="P",
    edgecolors="black",
    linewidths=1,
    label="Test Centroid",
)
ax1.scatter(
    test_tta_centroid_tsne[0],
    test_tta_centroid_tsne[1],
    s=250,
    c="red",
    marker="X",
    edgecolors="black",
    linewidths=1,
    label="Test Centroid with TTA",
)

ax1.set_title("t-SNE Projection")
ax1.set_xlabel("t-SNE Component 1")
ax1.set_ylabel("t-SNE Component 2")

# Zooming in a bit further to see the centroids better
x_min_train_tsne, x_max_train_tsne = np.percentile(
    train_embeddings_tsne[:, 0], [20, 80]
)
y_min_train_tsne, y_max_train_tsne = np.percentile(
    train_embeddings_tsne[:, 1], [20, 80]
)
x_min_test_tsne, x_max_test_tsne = np.percentile(test_embeddings_tsne[:, 0], [20, 80])
y_min_test_tsne, y_max_test_tsne = np.percentile(test_embeddings_tsne[:, 1], [20, 80])

x_max_tsne = max(x_max_train_tsne, x_max_test_tsne)
x_min_tsne = min(x_min_train_tsne, x_min_test_tsne)
y_max_tsne = max(y_max_train_tsne, y_max_test_tsne)
y_min_tsne = min(y_min_train_tsne, y_min_test_tsne)

ax1.set_xlim(x_min_tsne, x_max_tsne)
ax1.set_ylim(y_min_tsne, y_max_tsne)
fig.legend(bbox_to_anchor=(0.75, -0.1), loc="lower right", ncol=3)

# PCA plot on the right
ax2.scatter(
    test_embeddings_2d[:, 0],
    test_embeddings_2d[:, 1],
    alpha=0.5,
    label="Test Set without TTA",
    s=1.5,
    color="green",
)
ax2.scatter(
    test_embeddings_with_TTA_2d[:, 0],
    test_embeddings_with_TTA_2d[:, 1],
    alpha=0.5,
    label="Test Set with TTA",
    s=1.5,
    color="red",
)

# Plot centroids for PCA
ax2.scatter(
    train_centroid[0],
    train_centroid[1],
    s=250,
    c="blue",
    marker="P",
    edgecolors="black",
    linewidths=1,
    label="Train Centroid",
    alpha=0.9,
)
ax2.scatter(
    val_centroid[0],
    val_centroid[1],
    s=250,
    c="orange",
    marker="X",
    edgecolors="black",
    linewidths=1,
    label="Val Centroid",
    alpha=0.9,
)
ax2.scatter(
    test_centroid[0],
    test_centroid[1],
    s=250,
    c="green",
    marker="P",
    edgecolors="black",
    linewidths=1,
    label="Test Centroid",
    alpha=0.9,
)
ax2.scatter(
    test_tta_centroid[0],
    test_tta_centroid[1],
    s=250,
    c="red",
    marker="X",
    edgecolors="black",
    linewidths=1,
    label="Test Centroid with TTA",
    alpha=0.9,
)

ax2.set_title("UMAP Projection")
ax2.set_xlabel("UMAP Component 1")
ax2.set_ylabel("UMAP Component 2")

# Zooming in a bit further to see the centroids better
x_min_train_umap, x_max_train_umap = np.percentile(
    train_embeddings_umap[:, 0], [20, 80]
)
y_min_train_umap, y_max_train_umap = np.percentile(
    train_embeddings_umap[:, 1], [20, 80]
)
x_min_test_umap, x_max_test_umap = np.percentile(test_embeddings_umap[:, 0], [20, 80])
y_min_test_umap, y_max_test_umap = np.percentile(test_embeddings_umap[:, 1], [20, 80])

x_max_umap = max(x_max_train_umap, x_max_test_umap)
x_min_umap = min(x_min_train_umap, x_min_test_umap)
y_max_umap = max(y_max_train_umap, y_max_test_umap)
y_min_umap = min(y_min_train_umap, y_min_test_umap)

ax2.set_xlim(x_min_umap, x_max_umap)
ax2.set_ylim(y_min_umap, y_max_umap)

plt.tight_layout()
if save_plots:
    plt.savefig("figures/sets_TTA.jpg", dpi=300, bbox_inches="tight")
    plt.savefig("figures/sets_TTA.pdf", bbox_inches="tight")
plt.show()